In [101]:
import copy, sys, pandas as pd
import random
import time
import numpy as np
import subprocess

sys.path.append("../../")
sys.path.append("../../inferring")
sys.path.append("../../utils/DFA")
from importlib import reload


import inferring.Inferring as Inferring, inferring.InferringDFA as InferringDFA
import utils.automats.DFA
import utils.advice_systems.SRS as SRS
import utils.advice_systems.SRSconv_with_common as SRS_common

reload(SRS_common)

reload(SRS)
reload(Inferring)
reload(InferringDFA)
reload(utils.automats.DFA)

from inferring.Inferring import Inferring
from utils.automats.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS
from utils.advice_systems.SRSconv_with_common import SRSconv_with_common
from inferring.InferringDFA import InferringDFA

In [102]:
def run_learning_process(target, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(target)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [103]:
class Record:

    def __init__(self,
                 d1,
                 d2,
                 conv,
                 lstar_eq,
                 lstar_wa_eq,
                 common, 
                 target=None):
        self.d1 = d1
        self.d2 = d2
        self.conv = conv
        self.lstar_eq = lstar_eq
        self.lstar_wa_eq = lstar_wa_eq
        self.common = common
        self.target = target

    def print_record(self):
        print(
            f"|d1| = {self.d1}, |d2| = {self.d2}, |conv| = {self.conv},  eq = {self.lstar_eq}, eq_with_advice = {self.lstar_wa_eq}"
        )

    def get_complete_description(self):
        return self.target.print_complete_description()

    def print_target(self):
        print(self.target.print_complete_description())

In [148]:
import string

number_of_iteration = 10
max_number_of_states = 20
common_letters = [4, 3, 2, 1, 0]
unique_letters = [0, 1, 2, 3, 4]


results = []
i = 0
while i < number_of_iteration:
    random.seed(i)
    k = common_letters[i%len(common_letters)]  # number of common letters
    l = unique_letters[i%len(unique_letters)]  # number of unique letters
    input_signs = [a for a in string.ascii_lowercase[:l + k + l]]
    i += 1
    # Create two random DFAs
    dfa1 = DFA()
    dfa2 = DFA()
    dfa1.create_random_dfa(
        Q=random.randint(max_number_of_states // 2, max_number_of_states),
        input_signs=input_signs[:k + l],
    )
    dfa2.create_random_dfa(
        Q=random.randint(max_number_of_states // 2, max_number_of_states),
        input_signs=input_signs[-(k + l):],
    )

    # Run learning of the language of DFA to minimize dfa1 and dfa2
    d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))
    d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    while d1.Q < 2:
        dfa1 = DFA()
        dfa1.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs[:k + l],
        )
        d1, _, _ = run_learning_process(target=copy.deepcopy(dfa1))

    while d2.Q < 2:
        dfa2 = DFA()
        dfa2.create_random_dfa(
            Q=random.randint(max_number_of_states // 2, max_number_of_states),
            input_signs=input_signs[-(k + l):],
        )
        d2, _, _ = run_learning_process(target=copy.deepcopy(dfa2))

    # d1.print_dfa()
    # d2.print_dfa()

    conv_d = DFA()
    conv_d.create_convolution_with_common_letters(d1, d2)
    # conv_d.print_dfa()

    # Learn conv_dfa with advice, and store d - learned automaton
    start_t = time.time()
    d, lstar_with_advice, lstar_ex_with_advice = run_learning_process(
        target=conv_d, advice_system=SRS(), check_consistency=True)
    end_t = time.time()
    d.type = DFA.CONV_DFA_WITH_COMMON

    start_t = time.time()
    _, lstar, lstar_ex = run_learning_process(target=d,
                                              advice_system=None,
                                              check_consistency=False)
    end_t = time.time()

    results.append(
        Record(d1=d1.Q,
               d2=d2.Q,
               conv=d.Q,
               lstar_eq=lstar_ex,
               lstar_wa_eq=lstar_ex_with_advice,
               common=k / (l + k)))
    print(
        f"|d1.Q| = {d1.Q}, |d2.q| = {d2.Q}, |d.Q| = {d.Q}, l*ex = {lstar_ex} vs l*_WA_ex = {lstar_ex_with_advice}, COMMON letters = {k/(l+k)}"
    )

|d1.Q| = 16, |d2.q| = 19, |d.Q| = 261, l*ex = 67 vs l*_WA_ex = 67, COMMON letters = 1.0
|d1.Q| = 14, |d2.q| = 18, |d.Q| = 1, l*ex = 1 vs l*_WA_ex = 1, COMMON letters = 0.75
|d1.Q| = 20, |d2.q| = 13, |d.Q| = 256, l*ex = 73 vs l*_WA_ex = 106, COMMON letters = 0.5
|d1.Q| = 13, |d2.q| = 18, |d.Q| = 234, l*ex = 88 vs l*_WA_ex = 47, COMMON letters = 0.25
|d1.Q| = 12, |d2.q| = 17, |d.Q| = 204, l*ex = 100 vs l*_WA_ex = 87, COMMON letters = 0.0
|d1.Q| = 19, |d2.q| = 16, |d.Q| = 273, l*ex = 83 vs l*_WA_ex = 83, COMMON letters = 1.0
|d1.Q| = 19, |d2.q| = 11, |d.Q| = 200, l*ex = 55 vs l*_WA_ex = 64, COMMON letters = 0.75
|d1.Q| = 14, |d2.q| = 11, |d.Q| = 154, l*ex = 57 vs l*_WA_ex = 55, COMMON letters = 0.5
|d1.Q| = 13, |d2.q| = 16, |d.Q| = 208, l*ex = 93 vs l*_WA_ex = 69, COMMON letters = 0.25
|d1.Q| = 17, |d2.q| = 10, |d.Q| = 170, l*ex = 59 vs l*_WA_ex = 45, COMMON letters = 0.0


In [ ]:
results.sort(key=lambda x: (x.common, x.conv))
print(f"number of tests: {len(results)}")

number of tests: 10


In [143]:
def create_table(columns, data):
    columns = pd.MultiIndex.from_tuples(columns)
    df = pd.DataFrame(data, columns=columns)

    avg_eq_red = df[(('Reduction', 'EQ'))].mean()
    max_eq_red = df[(('Reduction', 'EQ'))].max()
    min_eq_red = df[(('Reduction', 'EQ'))].min()
    empty_row = pd.DataFrame([[""] * df.shape[1]], columns=df.columns)

    def set_params(row, params):
        for k, v in params:
            row[columns[k]] = v

    max_red_row = copy.deepcopy(empty_row)
    set_params(row=max_red_row, params=[(0, "Max reduction"), (-1, int(max_eq_red))])
    min_red_row = copy.deepcopy(empty_row)
    set_params(row=min_red_row, params=[(0, "Min reduction"), (-1, int(min_eq_red))])
    avg_red_row = copy.deepcopy(empty_row)
    set_params(row=avg_red_row, params=[(0, "Average reduction"), (-1, int(avg_eq_red))])

    rows = [empty_row, max_red_row, min_red_row, avg_red_row]
    for row in rows:
        df = pd.concat([df, row], ignore_index=True)

    df[('Reduction', 'EQ')] = df[('Reduction', 'EQ')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)
    df[('Common letters')] = df[('Common letters')].apply(lambda x: f'{x}%' if isinstance(x, int) else x)

    return df

In [150]:
lstar_eqs = [r.lstar_eq for r in results]
lstar_wa_eqs = [r.lstar_wa_eq for r in results]
red = (
    (np.array(lstar_eqs) - np.array(lstar_wa_eqs)) / np.array(lstar_eqs)) * 100
columns = [('Target language', 'conv(DFA1, DFA2)'), ('Common letters', ''), ('L*', 'EQ'), ('L* with advice', 'EQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.conv for r in results],
    columns[1]: [int(r.common*100) for r in results],
    columns[2]: lstar_eqs,
    columns[3]: lstar_wa_eqs,
    columns[4]: [int(r) for r in red]
}

df = create_table(columns=columns, data=data)
display(df)

,Target language,Common letters,L*,L* with advice,Reduction
,"conv(DFA1, DFA2)",,EQ,EQ,EQ
0,170,0%,59,45,23%
1,204,0%,100,87,13%
2,208,25%,93,69,25%
3,234,25%,88,47,46%
4,154,50%,57,55,3%
5,256,50%,73,106,-45%
6,1,75%,1,1,0%
7,200,75%,55,64,-16%
8,261,100%,67,67,0%
